In [ ]:
from simplepbi import token
from simplepbi import datasets
import pandas as pd
import time

In [ ]:
# Login variables
TENANT_ID = "[Tenant Id]"
power_bi_client_id = '[Client Id]' 
power_bi_secret = '[Secret Value]'

# Fabric Metrics Capacity App workspace and dataset id
capacity_id = "[Dedicated Capacity Id]"
workspace_id = "[Workspace Id]"
dataset_id = '[Semantic Model Id]'

In [ ]:
# Create token objetc and Dataset Object initialized by token
t = token.Token(TENANT_ID, power_bi_client_id, None, None, power_bi_secret, use_service_principal=True)
d = datasets.Datasets(t.token)

In [ ]:
# Define the operations schema (column names and data types)
schema = {
    'BillingType': str,
    'Status': str,
    'OperationStartTime': str,
    'OperationEndTime': str,
    'User': str,
    'Operation': str,
    'OperationID': str,
    'WorkspaceName': str,
    'Item': str,
    'ItemName': str,
    'TimepointCUs': float,
    'DurationInS': int,
    'TotalCUInS': float,
    'Throttling': int,
    'PercentageOfBaseCapacity': float,
    'capacityId': str,
    'Timepoint': str,
    'OperationType': str,
}

# Create an empty DataFrame with the defined schema
dataframe_schema = pd.DataFrame(columns=schema.keys())

# Set the types for background ops
df_bg = dataframe_schema.astype(schema)
# Set the types for interactive ops
df_int = dataframe_schema.astype(schema)

# Display the empty DataFrame
print(dataframe_schema)

In [ ]:
# Query to get TimePoints from Fabric Capacity Metrics and store it in pandas dataframe - date_day format yyyy-mm-dd
def get_timepoints(date_day, capacity_id):
    year = date_day.split("-")[0]
    month = date_day.split("-")[1]
    day = date_day.split("-")[2]
    dax_date = "DATE({},{},{})".format(year, month, day)

    query_tp= '''
    DEFINE
        MPARAMETER 'CapacityID' = "'''+ capacity_id +'''"
        VAR __DS0FilterTable =
            TREATAS ( { "'''+ capacity_id +'''" }, 'Capacities'[capacityId] )
        VAR __DS0Core =
            SUMMARIZECOLUMNS (
                ROLLUPADDISSUBTOTAL ( 'TimePoints'[TimePoint], "IsGrandTotalRowTotal" ),
                __DS0FilterTable,
                "CU_Limit", 'All Measures'[CU Limit],
                "SKU_CU_by_TimePoint__", 'All Measures'[SKU CU by TimePoint %],
                "xBackground__", 'All Measures'[xBackground %],
                "xInteractive__", 'All Measures'[xInteractive %],
                "xBackground___Preview", 'All Measures'[xBackground % Preview],
                "xInteractive___Preview", 'All Measures'[xInteractive % Preview]
            )
        VAR __DS0PrimaryWindowed =
            FILTER(__DS0Core, FORMAT([TimePoint], "yyyy-mm-dd") = FORMAT(''' + dax_date + ''', "yyyy-mm-dd") ) 

    EVALUATE
    __DS0PrimaryWindowed
    '''
    tp = d.execute_queries_in_group(workspace_id, dataset_id, query_tp, return_pandas=True)
    
    return tp

In [ ]:
# Get the timepoint by the date expected
tp = get_timepoints("2024-07-10", capacity_id)
for item in tp["TimePoint"]:
    print(item)

In [ ]:
# Function to get Background Operations of the Fabric Capacity Metrics App
def count_background_operations(date_day, capacity_id):
    tpoint = date_day.split("T")
    year = tpoint[0].split("-")[0]
    month = tpoint[0].split("-")[1]
    day = tpoint[0].split("-")[2]
    hour = tpoint[1].split(":")[0]
    minute = tpoint[1].split(":")[1]
    second = tpoint[1].split(":")[2]
    dax_datetime = "(DATE({}, {}, {}) + TIME({}, {}, {}))".format(year, month, day, hour, minute, second)
    
    dax_query= '''
    DEFINE
                        MPARAMETER 'CapacityID' = "''' + capacity_id + '''"
                        MPARAMETER 'TimePoint' = ''' + dax_datetime + '''

                        VAR varFilter_Capacity = TREATAS({"''' + capacity_id + '''"}, 'Capacities'[capacityId])
                        VAR varFilter_TimePoint = 
                            TREATAS(
                                { ''' + dax_datetime + ''' },
                                'TimePoints'[TimePoint]
                            )
                        VAR varTable_Details =
                            SUMMARIZECOLUMNS(
                                'TimePointBackgroundDetail'[OperationStartTime],
                                'TimePointBackgroundDetail'[OperationEndTime],
                                'TimePointBackgroundDetail'[Status],
                                'TimePointBackgroundDetail'[Operation],
                                'TimePointBackgroundDetail'[User],
                                'TimePointBackgroundDetail'[OperationId],
                                'TimePointBackgroundDetail'[Billing type],
                                'Items'[WorkspaceName],
                                'Items'[ItemKind],
                                'Items'[ItemName],
                                
                                varFilter_Capacity,
                                varFilter_TimePoint,
                                
                                "Timepoint CU (s)", SUM('TimePointBackgroundDetail'[Timepoint CU (s)]),
                                "Duration (s)", SUM('TimePointBackgroundDetail'[Duration (s)]),
                                "Total CU (s)", CALCULATE(SUM('TimePointBackgroundDetail'[Total CU (s)])),
                                "Throttling", CALCULATE(SUM('TimePointBackgroundDetail'[Throttling (s)])),
                                "% of Base Capacity", CALCULATE(SUM('TimePointBackgroundDetail'[% of Base Capacity]))
                            )
                                
                    VAR varTable_query =  SELECTCOLUMNS(
                        varTable_Details,
                        "BillingType", [Billing type],
                        "Status", [Status],
                        "OperationStartTime", [OperationStartTime],
                        "OperationEndTime", [OperationEndTime],
                        "User", [User],
                        "Operation", [Operation],
                        "OperationID", [OperationId],
                        "WorkspaceName", [WorkspaceName],
                        "Item", [ItemKind],
                        "ItemName", [ItemName],
                        "TimepointCUs", [Timepoint CU (s)],
                        "DurationInS", [Duration (s)],
                        "TotalCUInS", [Total CU (s)],
                        "Throttling", [Throttling],
                        "PercentageOfBaseCapacity", [% of Base Capacity]
                    )
                    
                    EVALUATE {COUNTROWS(varTable_query)}
                    '''
    df_value = d.execute_queries_in_group(workspace_id, dataset_id, dax_query, return_pandas=False)
    return df_value["results"][0]["tables"][0]["rows"][0]["[Value]"]

In [ ]:
# Function to get Background Operations of the Fabric Metrics Capacity App by delimited start and end rows with DAX WINDOW.

def dax_background_operations(date_day, capacity_id, row_start, row_end):
    tpoint = date_day.split("T")
    year = tpoint[0].split("-")[0]
    month = tpoint[0].split("-")[1]
    day = tpoint[0].split("-")[2]
    hour = tpoint[1].split(":")[0]
    minute = tpoint[1].split(":")[1]
    second = tpoint[1].split(":")[2]
    dax_datetime = "(DATE({}, {}, {}) + TIME({}, {}, {}))".format(year, month, day, hour, minute, second)
    
    dax_query= '''
    DEFINE
                        MPARAMETER 'CapacityID' = "''' + capacity_id + '''"
                        MPARAMETER 'TimePoint' = ''' + dax_datetime + '''

                        VAR varFilter_Capacity = TREATAS({"''' + capacity_id + '''"}, 'Capacities'[capacityId])
                        VAR varFilter_TimePoint = 
                            TREATAS(
                                { ''' + dax_datetime + ''' },
                                'TimePoints'[TimePoint]
                            )
                        VAR varTable_Details =
                            SUMMARIZECOLUMNS(
                                'TimePointBackgroundDetail'[OperationStartTime],
                                'TimePointBackgroundDetail'[OperationEndTime],
                                'TimePointBackgroundDetail'[Status],
                                'TimePointBackgroundDetail'[Operation],
                                'TimePointBackgroundDetail'[User],
                                'TimePointBackgroundDetail'[OperationId],
                                'TimePointBackgroundDetail'[Billing type],
                                'Items'[WorkspaceName],
                                'Items'[ItemKind],
                                'Items'[ItemName],
                                
                                varFilter_Capacity,
                                varFilter_TimePoint,
                                
                                "Timepoint CU (s)", SUM('TimePointBackgroundDetail'[Timepoint CU (s)]),
                                "Duration (s)", SUM('TimePointBackgroundDetail'[Duration (s)]),
                                "Total CU (s)", CALCULATE(SUM('TimePointBackgroundDetail'[Total CU (s)])),
                                "Throttling", CALCULATE(SUM('TimePointBackgroundDetail'[Throttling (s)])),
                                "% of Base Capacity", CALCULATE(SUM('TimePointBackgroundDetail'[% of Base Capacity]))
                            )
                                
                    VAR varTable_query =  SELECTCOLUMNS(
                        varTable_Details,
                        "BillingType", [Billing type],
                        "Status", [Status],
                        "OperationStartTime", [OperationStartTime],
                        "OperationEndTime", [OperationEndTime],
                        "User", [User],
                        "Operation", [Operation],
                        "OperationID", [OperationId],
                        "WorkspaceName", [WorkspaceName],
                        "Item", [ItemKind],
                        "ItemName", [ItemName],
                        "TimepointCUs", [Timepoint CU (s)],
                        "DurationInS", [Duration (s)],
                        "TotalCUInS", [Total CU (s)],
                        "Throttling", [Throttling],
                        "PercentageOfBaseCapacity", [% of Base Capacity]
                    )
                    
                    EVALUATE WINDOW(''' + row_start + ''', ABS,''' + row_end + ''', ABS, varTable_query)
                    '''
    df = d.execute_queries_in_group(workspace_id, dataset_id, dax_query, return_pandas=True)
    return df

In [ ]:
# Function to get Interactive Operations of the Fabric Capacity Metrics App
def count_interactive_operations(date_day, capacity_id):
    tpoint = date_day.split("T")
    year = tpoint[0].split("-")[0]
    month = tpoint[0].split("-")[1]
    day = tpoint[0].split("-")[2]
    hour = tpoint[1].split(":")[0]
    minute = tpoint[1].split(":")[1]
    second = tpoint[1].split(":")[2]
    dax_datetime = "(DATE({}, {}, {}) + TIME({}, {}, {}))".format(year, month, day, hour, minute, second)
    
    dax_query= '''
    DEFINE
                        MPARAMETER 'CapacityID' = "''' + capacity_id + '''"
                        MPARAMETER 'TimePoint' = ''' + dax_datetime + '''

                        VAR varFilter_Capacity = TREATAS({"''' + capacity_id + '''"}, 'Capacities'[capacityId])
                        VAR varFilter_TimePoint = 
                            TREATAS(
                                { ''' + dax_datetime + ''' },
                                'TimePoints'[TimePoint]
                            )
                        VAR varTable_Details =
                            SUMMARIZECOLUMNS(
                                'TimePointInteractiveDetail'[OperationStartTime],
                                'TimePointInteractiveDetail'[OperationEndTime],
                                'TimePointInteractiveDetail'[Status],
                                'TimePointInteractiveDetail'[Operation],
                                'TimePointInteractiveDetail'[User],
                                'TimePointInteractiveDetail'[OperationId],
                                'TimePointInteractiveDetail'[Billing type],
                                'Items'[WorkspaceName],
                                'Items'[ItemKind],
                                'Items'[ItemName],
                                
                                varFilter_Capacity,
                                varFilter_TimePoint,
                                
                                "Timepoint CU (s)", SUM('TimePointInteractiveDetail'[Timepoint CU (s)]),
                                "Duration (s)", SUM('TimePointInteractiveDetail'[Duration (s)]),
                                "Total CU (s)", CALCULATE(SUM('TimePointInteractiveDetail'[Total CU (s)])),
                                "Throttling", CALCULATE(SUM('TimePointInteractiveDetail'[Throttling (s)])),
                                "% of Base Capacity", CALCULATE(SUM('TimePointInteractiveDetail'[% of Base Capacity]))
                            )
                                
                    VAR varTable_query = SELECTCOLUMNS(
                        varTable_Details,
                        "BillingType", [Billing type],
                        "Status", [Status],
                        "OperationStartTime", [OperationStartTime],
                        "OperationEndTime", [OperationEndTime],
                        "User", [User],
                        "Operation", [Operation],
                        "OperationID", [OperationId],
                        "WorkspaceName", [WorkspaceName],
                        "Item", [ItemKind],
                        "ItemName", [ItemName],
                        "TimepointCUs", [Timepoint CU (s)],
                        "DurationInS", [Duration (s)],
                        "TotalCUInS", [Total CU (s)],
                        "Throttling", [Throttling],
                        "PercentageOfBaseCapacity", [% of Base Capacity]
                    )
                    
                    EVALUATE {COUNTROWS(varTable_query)}
                    '''
    df_value = d.execute_queries_in_group(workspace_id, dataset_id, dax_query, return_pandas=False)
    return df_value["results"][0]["tables"][0]["rows"][0]["[Value]"]

In [ ]:
# Function to get Interactive Operations of the Fabric Metrics Capacity App by delimited start and end rows with DAX WINDOW.

def dax_interactive_operations(date_day, capacity_id, row_start, row_end):
    tpoint = date_day.split("T")
    year = tpoint[0].split("-")[0]
    month = tpoint[0].split("-")[1]
    day = tpoint[0].split("-")[2]
    hour = tpoint[1].split(":")[0]
    minute = tpoint[1].split(":")[1]
    second = tpoint[1].split(":")[2]
    dax_datetime = "(DATE({}, {}, {}) + TIME({}, {}, {}))".format(year, month, day, hour, minute, second)
    
    dax_query= '''
    DEFINE
                        MPARAMETER 'CapacityID' = "''' + capacity_id + '''"
                        MPARAMETER 'TimePoint' = ''' + dax_datetime + '''

                        VAR varFilter_Capacity = TREATAS({"''' + capacity_id + '''"}, 'Capacities'[capacityId])
                        VAR varFilter_TimePoint = 
                            TREATAS(
                                { ''' + dax_datetime + ''' },
                                'TimePoints'[TimePoint]
                            )
                        VAR varTable_Details =
                            SUMMARIZECOLUMNS(
                                'TimePointInteractiveDetail'[OperationStartTime],
                                'TimePointInteractiveDetail'[OperationEndTime],
                                'TimePointInteractiveDetail'[Status],
                                'TimePointInteractiveDetail'[Operation],
                                'TimePointInteractiveDetail'[User],
                                'TimePointInteractiveDetail'[OperationId],
                                'TimePointInteractiveDetail'[Billing type],
                                'Items'[WorkspaceName],
                                'Items'[ItemKind],
                                'Items'[ItemName],
                                
                                varFilter_Capacity,
                                varFilter_TimePoint,
                                
                                "Timepoint CU (s)", SUM('TimePointInteractiveDetail'[Timepoint CU (s)]),
                                "Duration (s)", SUM('TimePointInteractiveDetail'[Duration (s)]),
                                "Total CU (s)", CALCULATE(SUM('TimePointInteractiveDetail'[Total CU (s)])),
                                "Throttling", CALCULATE(SUM('TimePointInteractiveDetail'[Throttling (s)])),
                                "% of Base Capacity", CALCULATE(SUM('TimePointInteractiveDetail'[% of Base Capacity]))
                            )
                                
                    VAR varTable_query = SELECTCOLUMNS(
                        varTable_Details,
                        "BillingType", [Billing type],
                        "Status", [Status],
                        "OperationStartTime", [OperationStartTime],
                        "OperationEndTime", [OperationEndTime],
                        "User", [User],
                        "Operation", [Operation],
                        "OperationID", [OperationId],
                        "WorkspaceName", [WorkspaceName],
                        "Item", [ItemKind],
                        "ItemName", [ItemName],
                        "TimepointCUs", [Timepoint CU (s)],
                        "DurationInS", [Duration (s)],
                        "TotalCUInS", [Total CU (s)],
                        "Throttling", [Throttling],
                        "PercentageOfBaseCapacity", [% of Base Capacity]
                    )
                    
                    EVALUATE WINDOW(''' + row_start + ''', ABS,''' + row_end + ''', ABS, varTable_query)
                    '''
    df = d.execute_queries_in_group(workspace_id, dataset_id, dax_query, return_pandas=True)
    return df

In [ ]:
# Concat all background data looping timepoints and splitting each one by 1.000.000 values due to API limitations. 
# Acording with the column count it's 66666 rows.

conta = 0
tp_looper = tp[['TimePoint']].loc[~tp['xBackground__'].isna()]['TimePoint']
# Looping timepoints for the date
for item in tp_looper:    
    print("Timepoint ", str(conta), " from ", str(len(tp_looper)))
    bg_ops = count_background_operations(item, capacity_id)
    # Sleep for limit of queries per minute if None, wait and retry
    if(bg_ops == None):            
        print ("Waiting a minute to retry for possible API limitations. Too many requests issue.")
        time.sleep(60) 
        bg_ops = count_background_operations(item, capacity_id)
    max_values = 66666
    bg_ops_len = int(bg_ops/max_values)+1
    # Looping 66666 rows due to API limits
    for n in range(bg_ops_len):
        start = 0    
        if(n==0):
            start = "1"
        else:
            start = str(max_values*n)
        end = str(max_values*(n+1))
        # Sleep for limit of queries per minute if None, wait and retry
        df = None
        df = dax_background_operations(item, capacity_id, start, end)
        if(df is None):            
            print ("Waiting a minute to retry for possible API limitations. Too many requests issue.")
            time.sleep(60) 
            df = dax_background_operations(item, capacity_id, start, end)
        print("Loop in timepoint ", str(n), " from ", str(bg_ops_len))
        df["capacityId"]=capacity_id
        df["Timepoint"]=item
        df["OperationType"]="Background"
        df_bg = pd.concat([df_bg, df] , ignore_index=True )    
    conta = conta + 1
        
df_bg.head()

In [ ]:
# Concat all interactive data looping timepoints and splitting each one by 1.000.000 values due to API limitations. 
# Acording with the column count it's 66666 rows.

conta = 0
tp_looper = tp[['TimePoint']].loc[~tp['xInteractive__'].isna()]['TimePoint']
for item in tp_looper:
    print("Timepoint ", str(conta), " from ", str(len(tp_looper)))
    int_ops = count_interactive_operations(item, capacity_id)
    # Sleep for limit of queries per minute if None, wait and retry
    if(int_ops == None):            
        print ("Waiting a minute to retry for possible API limitations. Too many requests issue.")
        time.sleep(60) 
        int_ops = count_interactive_operations(item, capacity_id)
    max_values = 66666
    int_ops_len = int(int_ops/max_values if int_ops != None else -1)+1
    for n in range(int_ops_len):
        start = 0    
        if(n==0):
            start = "1"
        else:
            start = str(max_values*n)
        end = str(max_values*(n+1))
        df = None
        df = dax_interactive_operations(item, capacity_id, start, end)
        # Sleep for limit of queries per minute if None, wait and retry
        if(df is None):            
            print ("Waiting a minute to retry for possible API limitations. Too many requests issue.")
            time.sleep(60) # Sleep for limit of queries per minute
            df = dax_interactive_operations(item, capacity_id, start, end)
        print("Loop in timepoint ", str(n), " from ", str(int_ops_len))
        df["capacityId"]=capacity_id
        df["Timepoint"]=item
        df["OperationType"]="Interactive"
        df_int = pd.concat([df_int, df] , ignore_index=True )
    conta = conta + 1
        
df_int.head()

In [ ]:
# Get Fabric items involved in operations
df_item = d.execute_queries_in_group(workspace_id, dataset_id, "EVALUATE Items", return_pandas=True)

In [ ]:
# Tables generated
'''
TimePoints = tp
Backgrounds ops = df_bg
Interative = df_int
Items = df_item
'''